## Initializing opencv haarcascade face detection network
https://github.com/opencv/opencv/tree/master/data/haarcascades
explained in: https://www.youtube.com/watch?v=7IFhsbfby9s&t=300s (or gitHub)

In [185]:
import cv2
import os

In [186]:
def convertImg(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

def faceNetLocalize(img, **kwargs):
    scaleFactor = kwargs.get('scaleFactor', 1.1) #between 1.05 (quality) and 1.4 (speed) recommended (scale of the faces we search for)
    minNeighbors = kwargs.get('minNeighbors', 4) #between 3 (quantity) and 6 (quality) recommended
    minSize = kwargs.get('minSize', (10, 10)) #min size of a face in the picture
    faceNet = kwargs.get('faceNet', init_faceNet())
    
    img_cvt = convertImg(img)
    return faceNet.detectMultiScale3(img_cvt, scaleFactor=scaleFactor, minNeighbors=minNeighbors, minSize=minSize, outputRejectLevels = True)

def init_faceNet(**kwargs):
    path = kwargs.get('path', 'haarcascade_frontalface_default.xml')
    return cv2.CascadeClassifier(path)


In [187]:
#TODO: testing with different model types
#for example: eye model + larger bounding box towards the bottom

## Mask classifier

foundation: https://www.tensorflow.org/tutorials/load_data/images

In [188]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import MaxPool2D, Conv2D, Input, Dense, Flatten, AveragePooling2D, Dropout
import tensorflow.keras.layers as lays
from tensorflow.keras.layers.experimental.preprocessing import Rescaling, RandomContrast, RandomFlip, RandomRotation
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import Sequential, losses as lfs
from tensorflow.keras.optimizers import Adam
import keras_tuner as kt
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import utils

#### Data augmentation
https://www.tensorflow.org/tutorials/images/data_augmentation

In [189]:
augmentation = Sequential([
  RandomFlip("horizontal"),
  RandomRotation(0.4),
  RandomContrast(0.5)
])

##### variables:

In [190]:
batch_size = 32
img_size = (180, 180)
img_size_vgg = (224, 224)
epochs = 11
checkpoint_path = "mask_model/weights.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
imgs_path = os.path.join('..', 'img')
num_classes = 3

correct_usage =  'correct usage: \n' + 'predict([path to image], \'category\' \n' + 'predict([path to image], \'probabilities\' \n' + 'predict([path to image], \'detection\' \n' + 'predict(\'live_detection\')'

##### loading dataset:

In [191]:
def load_dataset(**kwargs):
    imgs_path = kwargs.get('imgs_path', os.path.join('..', 'img'))
    img_size = kwargs.get('img_size', (180, 180))
    batch_size = kwargs.get('batch_size', 32)

    ##load images/ labels from directory
    valid_images = [".jpg",".png",".jpeg",".JPG"]
    x=[]
    y=[]
    
    for root, dirs, files in os.walk(imgs_path):
        for filename in files:
            end = os.path.splitext(filename)[1]
            if end.lower() not in valid_images:
                continue
            image = load_img(os.path.join(root, filename), target_size=img_size)
            image = img_to_array(image)
            
            label = os.path.join(root, filename).split(os.path.sep)[-2]
            
            x.append(image)
            y.append(label)
    x = np.array(x, dtype="float32")
    y = np.array(y)

    ## convert labels to ints from 0 ... len(labels)-1
    labels = []
    for i in range(len(y)):
        try:
            j = labels.index(y[i])
        except:
            labels.append(y[i])
            j = labels.index(y[i])
        y[i] = j
    y.astype(int)
    
    ## split dataset
    trainX, testX, trainY, testY = train_test_split(x, y, test_size=0.2, random_state=3)

    ## one-hot encoding
    trainY = utils.to_categorical(trainY, num_classes)
    testY = utils.to_categorical(testY, num_classes)

    ## data augmentation
    datagen = ImageDataGenerator(
        featurewise_center=True,
        featurewise_std_normalization=True,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        validation_split=0.2
    )

    ## merge xs and ys
    train_batches = datagen.flow(trainX, trainY, batch_size=32, subset='training')
    test_batches  = datagen.flow(trainX, trainY, batch_size=32, subset='validation')
    
    return train_batches, test_batches, labels, testX, testY, trainX, trainY

### Model selection & Training

In [192]:
#Hypermodels
#https://www.analyticsvidhya.com/blog/2021/06/create-convolutional-neural-network-model-and-optimize-using-keras-tuner-deep-learning/
#https://www.tensorflow.org/tutorials/keras/keras_tuner

def basic_model_builder(hp):
  
    basic_model = Sequential([
        Rescaling(1. /255),
        augmentation,
        Conv2D(filters=hp.Int('c1_filter', min_value=32, max_value=256, step=16), kernel_size=hp.Choice('c1_kernel', values=[3,5]), activation='relu'),
        AveragePooling2D(pool_size=(7,7)),
        Flatten(name="flatten"),
        Dense(units=hp.Int('d1_units', min_value=32, max_value=512, step=32), activation="relu"),
        Dropout(0.2),#drops small confidences
        Dense(num_classes, activation="softmax")
    ])

    basic_model.compile(optimizer=Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])), loss=lfs.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
    return basic_model

def small_model_builder(hp):
  
    small_model = Sequential([
        Rescaling(1. /255),
        augmentation,
        Conv2D(filters=hp.Int('c1_filter', min_value=32, max_value=256, step=16), kernel_size=hp.Choice('c1_kernel', values=[3,5]), activation='relu'),
        Conv2D(filters=hp.Int('c2_filter', min_value=32, max_value=256, step=16), kernel_size=hp.Choice('c2_kernel', values=[3,5]), activation='relu'),
        MaxPool2D(pool_size=(3,3)),
        Flatten(name="flatten"),
        Dense(units=hp.Int('d1_units', min_value=32, max_value=512, step=32), activation="relu"),
        Dropout(0.2),#drops small confidences
        Dense(num_classes, activation="softmax")
    ])

    small_model.compile(optimizer=Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])), loss=lfs.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
    return small_model


def tune_model(model_builder, xs, ys):
    #tuner = kt.RandomSearch(model_builder, objective='val_accuracy', max_trials=5)
    tuner = kt.RandomSearch(kt.applications.HyperResNet(input_shape=(180, 180, 3), classes=2), objective='val_loss', max_trials=5)
    tuner = kt.Hyperband(model_builder, objective='val_accuracy', max_epochs=10, factor=3)
    tuner.search(xs, ys, epochs=50, validation_split=0.2)
    return tuner.get_best_hyperparameters(num_trials=1)[0], tuner

In [193]:
def select_model(model_name, **kwargs):
    num_labels = kwargs.get('num_classes', num_classes)

    basic_model = Sequential([ 
    Rescaling(1. /255),
    Conv2D(32, (3,3), activation='relu', input_shape=(180,180,3)),
    AveragePooling2D(pool_size=(7,7)),
    Flatten(name="flatten"),
    Dense(128, activation="relu"),
    Dropout(0.5),#drops small confidences
    Dense(num_labels, activation="softmax")
    ])

    small_model = Sequential([ 
        Rescaling(1. /255),
        Conv2D(filters=128, kernel_size=(5,5), activation='relu', input_shape=(180,180,3)),
        Conv2D(filters=128, kernel_size=(5,5), activation='relu'),
        MaxPool2D(pool_size=(3,3)),
        Flatten(name="flatten"),
        Dense(units=224, activation="relu"),
        Dropout(0.5),#drops small confidences
        Dense(num_labels, activation="softmax")
        ])

    vgg_small_model=Sequential()

    vgg_small_model.add(Conv2D(64,(3,3),activation='relu',input_shape=(180,180,3)))
    vgg_small_model.add(MaxPool2D(2,2))
    vgg_small_model.add(Conv2D(64,(3,3),activation='relu'))
    vgg_small_model.add(MaxPool2D(2,2))
    vgg_small_model.add(Conv2D(128,(3,3),activation='relu'))
    vgg_small_model.add(MaxPool2D(2,2))
    vgg_small_model.add(Conv2D(128,(3,3),activation='relu'))
    vgg_small_model.add(MaxPool2D(2,2))
    vgg_small_model.add(Flatten())
    vgg_small_model.add(Dropout(0.5))
    vgg_small_model.add(Dense(120,activation='relu'))
    vgg_small_model.add(Dense(3,activation='softmax'))

    vgg_model = Sequential([
        Rescaling(1. /255),
        Conv2D(input_shape=(224,224,3), filters=64, kernel_size=(3,3), padding="same", activation="relu", strides=(1,1)), 
        Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"),
        MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
        Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"),
        Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"),
        MaxPool2D(pool_size=(2, 2), strides=(2)),
        Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"),
        Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"),
        Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"),
        MaxPool2D(pool_size=(2, 2), strides=(2)),
        Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"),
        Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"),
        Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"),
        MaxPool2D(pool_size=(2, 2), strides=(2)),
        Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"),
        Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"),
        Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"),
        MaxPool2D(pool_size=(2, 2), strides=(2)),
        Flatten(),
        Dense(units=4096, activation="relu"),
        Dense(units=4096, activation="relu"),
        Dense(units=num_labels, activation="softmax")
    ])

    if model_name == 'basic_model':
        return basic_model
    
    if model_name == 'small_model':
        return small_model

    if model_name == 'vgg_model':
        return vgg_model
    
    if model_name == 'vgg_small_model':
        return vgg_small_model

about the VGG-model: https://neurohive.io/en/popular-networks/vgg16/

In [194]:
def train_model(model, train_ds, val_ds, **kwargs):
    epochs = kwargs.get('epochs', 10)

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    history = model.fit(train_ds, validation_data=val_ds, epochs=epochs)
    return history

#### Testing/Evaluation (mask):
https://www.tensorflow.org/guide/keras/train_and_evaluate

In [195]:
def evaluate_model(x_test, y_test, model):
    results = model.evaluate(x_test, y_test, batch_size=32)
    print(results)
    
    y_pred_confidences = model.predict(x_test)
    y_pred = [np.argmax(cs) for cs in y_pred_confidences]
    print(classification_report(y_test, y_pred))


##### Predictions:

In [196]:
def load_model(**kwargs):
    checkpoint_path = kwargs.get('checkpoint_path', "mask_model/weights.ckpt")
    model = kwargs.get('model', select_model('basic_model'))
    model.load_weights(checkpoint_path)
    return model

In [197]:
def maskPredict(img, labels):
    pred = model.predict(img[None])
    label_index = np.argmax(pred)
    return labels[label_index], pred[0][label_index]


#mode can be 'category', 'probabilities', 'detection', 'live_detection'
def predict(mode, **kwargs):
    img_path = kwargs.get('img_path', None)
    model = kwargs.get('model', load_model())
    if mode=='live_detection':
        live_det()
        return
    
    if img_path is None:
        print(correct_usage)
        return

    #load_img
    img = load_img(img_path, target_size = img_size)
    img = img_to_array(img)
    
    if mode=='detection':
        detect(img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        return
        
    if mode=='category':
        label, confidence = maskPredict(img)
        return label
        
    if mode=='probabilities':
        #TODO: schöneres Format
        return model.predict(img[None])

    else:
        print(correct_usage)

def detect(img):
    faceLocs, rejectLevels, confidences = faceNetLocalize(img)
        
    for (x, y, w, h) in faceLocs:
        #crop image and predict label of cropped image
        img_crop = img[y:y+h, x:x+w]
        label, confidence_mask = maskPredict(img)
        #show label/ bounding box on image
        cv2.putText(img, f"{label}, confidence:{confidence_mask}", (x+w-30, y+h), cv2.FONT_HERSHEY_PLAIN, 1.0, cv2.CV_RGB(0,255,0), 2.0) 
        cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
    
    cv2.imshow('live_output', img)

def live_det():
    #TODO: errorhandling for camera
    
    wait_time = 10 #time in ms to wait before refreshing feed
    camera = cv2.VideoCapture(0) #Input value might differ on different systems
    
    while(True):
        _, img = camera.read()

        detect(img)

        #wait for ESC or q
        if (cv2.waitKey(wait_time) & 0xFF) in [27, ord('q')]: 
            break

    camera.release()
    return 'live_output'



In [198]:
train_ds, test_ds, labels, testX, testY, trainX, trainY = load_dataset()
#bhps, tuner = tune_model(basic_model_builder, x_test, y_test)

In [199]:
model = select_model('vgg_small_model')
#model = tuner.hypermodel.build(bhps)
history = train_model(model, train_ds, test_ds, epochs=15)
evaluate_model(testX, testY, model)

C:\Users\Julie\anaconda3\lib\site-packages\keras_preprocessing\image\image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
C:\Users\Julie\anaconda3\lib\site-packages\keras_preprocessing\image\image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/15
8/8 [==============================] - 11s 1s/step - loss: 24.7669 - accuracy: 0.3633 - val_loss: 1.4504 - val_accuracy: 0.2787
Epoch 2/15
8/8 [==============================] - 10s 1s/step - loss: 1.2100 - accuracy: 0.4204 - val_loss: 1.1712 - val_accuracy: 0.3934
Epoch 3/15
8/8 [==============================] - 11s 1s/step - loss: 1.0612 - accuracy: 0.4612 - val_loss: 0.8167 - val_accuracy: 0.6721
Epoch 4/15
8/8 [==============================] - 12s 1s/step - loss: 0.7761 - accuracy: 0.6816 - val_loss: 0.6202 - val_accuracy: 0.7541
Epoch 5/15
8/8 [==============================] - 11s 1s/step - loss: 0.5632 - accuracy: 0.7592 - val_loss: 0.4734 - val_accuracy: 0.8033
Epoch 6/15
8/8 [==============================] - 10s 1s/step - loss: 0.4632 - accuracy: 0.7837 - val_loss: 0.4139 - val_accuracy: 0.8361
Epoch 7/15
8/8 [==============================] - 11s 1s/step - loss: 0.4409 - accuracy: 0.8245 - val_loss: 0.3495 - val_accuracy: 0.9016
Epoch 8/15
8/8 [=================

ValueError: Classification metrics can't handle a mix of multilabel-indicator and multiclass targets

In [ ]:
#img_path='../img/ffp2/IMG_1596_(657, 421, 1426, 1426).png'
#img_path='../img/no_mask/IMG_1323.JPG_(1025, 427, 984, 984).png'
#img_path='../img/op_mask/IMG_1337.JPG_(878, 710, 954, 954).png'
#img_path='../img/other_mask/IMG_1327.JPG_(863, 476, 1097, 1097).png'

#predict(img_path, 'category', model=model, labels=labels)